# import packages

In [1]:
# more common imports
import pandas as pd
import numpy as np
from collections import Counter
import re

# languange processing imports
import nltk
import string
from gensim.corpora import Dictionary
# preprocessing imports
from sklearn.preprocessing import LabelEncoder
from pprint import pprint
# model imports
from gensim.utils import simple_preprocess

from gensim.models import CoherenceModel, Phrases, phrases
from gensim.models.doc2vec import LabeledSentence
from gensim.models.doc2vec import TaggedDocument
from gensim.models.doc2vec import Doc2Vec
from sklearn.preprocessing import normalize

import warnings
warnings.filterwarnings("ignore",category=FutureWarning)
warnings.filterwarnings("ignore",category=DeprecationWarning)

c:\users\nintyfive\appdata\local\programs\python\python35\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# read tables from mysql

In [ ]:
printable = set(string.printable)
s = 'Ti\u00ebsto'
list(filter(lambda x: x in printable, s))

In [2]:
# read lyrics.csv
data = pd.read_csv('lyrics.csv', names=['singer', 'song', 'lyrics'])

In [3]:
def preprocess(text):
    try:
        return simple_preprocess(text)
    except Exception as e:
        print(str(e))
        return np.nan

In [ ]:
#lyrics['singer'] = list(map(simple_preprocess, lyrics.singer))
#lyrics['song'] = list(map(simple_preprocess, lyrics.singer))
data['lyrics'] = list(map(preprocess, data.lyrics.values))
data = data[~data['lyrics'].isna()]

# constrcut doc2vec model

In [18]:
def infer_w2v_features(d2v_model, sentence):
    #vector = d2v_model.docvecs[i]
        
    inferred_vector = d2v_model.infer_vector(sentence)
    return inferred_vector

In [35]:
from gensim.test.utils import get_tmpfile
train_corpus = list(map(lambda i:TaggedDocument(data.loc[i].lyrics,['doc%d' % i]), data.index))

model = Doc2Vec(min_count=1, window=10, vector_size=100, sample=1e-4, negative=5, workers=8, epochs=200)
model.build_vocab(train_corpus)

%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

#train_data3['w2v_features'] = list(map(lambda i:
#                                      get_w2v_features(model, i),
#                                      range(0, train_data3.shape[0])))
fname = get_tmpfile("my_doc2vec_model")

model.save(fname)


Wall time: 6min 58s


In [36]:
fname = get_tmpfile("my_doc2vec_model")
model = Doc2Vec.load(fname)

In [41]:
ranks = []
for index, doc_id in enumerate(data.index):
    inferred_vector = model.infer_vector(train_corpus[index].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index('doc%d' % doc_id)
    ranks.append(rank)
    
import collections
collections.Counter(ranks) 
    

Counter({0: 11850,
         1: 17,
         2: 2,
         3: 3,
         4: 4,
         5: 2,
         6: 5,
         12: 2,
         13: 1,
         16: 2,
         18: 2,
         19: 2,
         20: 2,
         22: 3,
         24: 1,
         6938: 1})

In [51]:
print(data[data['song'] == 'Shattered'].lyrics)

6343    [shattered, shattered, love, and, hope, and, s...
Name: lyrics, dtype: object


In [55]:
lyric = 'Im a shattered  yeah  Pride and joy and greed and sex Thats what makes our town the best Pride and joy and dirty dreams'
inferred_vector = model.infer_vector(lyric.split(' '))
sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
rank = [docid for docid, sim in sims].index('doc%d' % 6343)

In [56]:
rank

0

In [21]:
import pickle
with open('store_ranks.npy', 'wb') as writer:
    pickle.dump(ranks, writer)
    
with open('store_ranks.npy', 'rb') as reader:
    ranks3 = pickle.load(reader)
collections.Counter(ranks3)

Counter({0: 6872, 1: 3})

In [94]:
# read lyrics.csv
data = pd.read_csv('lyrics.csv', names=['singer', 'song', 'lyrics'])
data = data[~data['lyrics'].isna()]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

count_vect = CountVectorizer().fit(data.lyrics.values)
words_count = count_vect.transform(data.lyrics.values)

tf_transformer = TfidfTransformer().fit(words_count)
words_tfidf = tf_transformer.transform(words_count)

In [81]:
lyric = 'who neglected you Owes a million dollars tax And your fathers still perfecting ways of making sealing wax'
words_count = count_vect.transform([lyric])
search_tfidf = tf_transformer.transform(words_count)

In [82]:
similar_result = pd.DataFrame(index=data.index, columns=['sim'])

In [83]:
for pr_index, pr in data.iterrows():
    reviewed_iloc = data.index.get_loc(pr_index)
    #print(reviewed_iloc)
    if len(np.dot(search_tfidf[0], words_tfidf[reviewed_iloc].transpose()).tocoo().data) == 1: 
        similar_score = np.dot(search_tfidf[0], words_tfidf[reviewed_iloc].transpose()).tocoo().data[0] #if len(np.dot(words_tfidf[pr_iloc], words_tfidf[i].transpose()).tocoo().data) == 1]
        similar_result.loc[pr_index, 'sim'] = similar_score

In [85]:
similar_result.sort_values(by=['sim'], ascending=False)

,sim
10345,0.274625
6344,0.272511
6722,0.172306
4709,0.150619
8530,0.145479
2387,0.139094
8988,0.138011
2557,0.116804
6120,0.11586
9268,0.110611


# trying fuzzy string comparison

In [87]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

c:\users\nintyfive\appdata\local\programs\python\python35\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
choices = ["Atlanta Falcons", "New York Jets", "New York Giants", "Dallas Cowboys"]
process.extract("new york jets", choices, limit=2)

In [100]:
data.loc[6343]

singer                                   The Rolling Stones
song                                              Shattered
lyrics    Shattered  shattered Love and hope and sex and...
Name: 6343, dtype: object

In [102]:
fuzz.ratio("Im a shattered  yeah  Pride and joy and greed and sex Thats what makes our town the best Pride and joy and dirty dreams", 'Shattered  shattered Love and hope and sex and dreams Are still surviving on the street Look at me  Im in tatters Im a shattered Shattered  Friends are so alarming And my lovers never charming Lifes just a cocktail party on the street Big Apple People dressed in plastic bags Directing traffic Some kind of fashion Shattered  Laughter  joy  and loneliness and sex and sex and sex and sex Look at me  Im in tatters Im a shattered Shattered  All this chitter-chatter  chitter-chatter  chitter-chatter about Shmatta  shmatta  shmatta  I cant give it away on 7th Avenue This towns been wearing tatters shattered  shattered Work and work for love and sex Aint you hungry for success  success  success  success Does it matter? Shattered Does it matter? Im shattered Shattered  Ahhh  look at me  Im a shattered Im a shattered Look at me  Im a shattered  yeah  Pride and joy and greed and sex Thats what makes our town the best Pride and joy and dirty dreams and still surviving on the street And look at me  Im in tatters  yeah Ive been battered  what does it matter Does it matter  uh-huh Does it matter  uh-huh  Im a shattered Dont you know the crime rate is going up  up  up  up  up To live in this town you must be tough  tough  tough  tough  tough! You got rats on the west side Bed bugs uptown What a mess this towns in tatters Ive been shattered My brains been battered  splattered all over Manhattan  Uh-huh  this towns full of money grabbers Go ahead  bite the Big Apple  dont mind the maggots  huh Shadoobie  my brains been battered My friends they come around they Flatter  flatter  flatter  flatter  flatter  flatter  flatter Pile it up  pile it high on the platter')

5

In [113]:
import regex
result = regex.match('(Im a yeah  Pride and joy and greed and sex Thats what makes our town the best Pride and joy and dirty dreams){e<=1}', 'Shattered  shattered Love and hope and sex and dreams Are still surviving on the street Look at me  Im in tatters Im a shattered Shattered  Friends are so alarming And my lovers never charming Lifes just a cocktail party on the street Big Apple People dressed in plastic bags Directing traffic Some kind of fashion Shattered  Laughter  joy  and loneliness and sex and sex and sex and sex Look at me  Im in tatters Im a shattered Shattered  All this chitter-chatter  chitter-chatter  chitter-chatter about Shmatta  shmatta  shmatta  I cant give it away on 7th Avenue This towns been wearing tatters shattered  shattered Work and work for love and sex Aint you hungry for success  success  success  success Does it matter? Shattered Does it matter? Im shattered Shattered  Ahhh  look at me  Im a shattered Im a shattered Look at me  Im a shattered  yeah  Pride and joy and greed and sex Thats what makes our town the best Pride and joy and dirty dreams and still surviving on the street And look at me  Im in tatters  yeah Ive been battered  what does it matter Does it matter  uh-huh Does it matter  uh-huh  Im a shattered Dont you know the crime rate is going up  up  up  up  up To live in this town you must be tough  tough  tough  tough  tough! You got rats on the west side Bed bugs uptown What a mess this towns in tatters Ive been shattered My brains been battered  splattered all over Manhattan  Uh-huh  this towns full of money grabbers Go ahead  bite the Big Apple  dont mind the maggots  huh Shadoobie  my brains been battered My friends they come around they Flatter  flatter  flatter  flatter  flatter  flatter  flatter Pile it up  pile it high on the platter')

In [ ]:
choices = data.lyrics.values
%time process.extract("Im a shattered  yeah  Pride and joy and greed and sex Thats what makes our town the best Pride and joy and dirty dreams", choices, limit = 5)

In [34]:
Counter(reviewers_semantic_sim['rchande'].isnull())

Counter({False: 8602})